# YOLO Object Tracking Test 

This notebook explores the possiblity of making use of the YOLO algorithm to perform tracking of the mouse moving within a maze. 

YOLO is a machine learning model designed to perform real-time object localization and detection and is widely used for a variety of computer vision based tasks. It's well known for its impressive speed and accuracy, and ease of use. 

In this experiment, we use YOLO v8.2 to perform image localization. This is the process of identifying a set of two points on the image, known as the bounding box, which surround an object within the image. 

In [1]:
import numpy as np 
import cv2 
from tqdm import trange
import matplotlib.pyplot as plt 
import ultralytics 
import os 
from typing import Optional
from tqdm import tqdm 
import torch
import wandb
from wandb.integration.ultralytics import add_wandb_callback 
from collections import defaultdict

import cv2
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors

VIDEO_PATH = "/Users/henrywilliams/Downloads/videos/maze_test.mp4" 
TRAIN = False

# wandb.login()
# wandb.init(
#     project="mice-maze",
#     job_type="training",
#     entity="lainon",
#     name="yolov8 for mouse tracking"
# )
yolo = YOLO()
cap = cv2.VideoCapture(VIDEO_PATH)
n_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

/Users/henrywilliams/Documents/programming/python/mice-maze-rewrite/yolo-test/.yolo-test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: WARNING This integration is tested and supported for ultralytics v8.0.238 and below.
wandb: WARNING             Please report any issues to https://github.com/wandb/wandb/issues with the tag `yolov8`.


# Create a Dataset

The following code cell takes 250 random frames from the video capture, and saves them to the `./frames/` directory. These frames are later annotated and used to fine-tune the YOLO model. To change where the frames are saved, modify the `OUTPUT_DIR` variable, and in order to change the number of samples captured, modify the `TOTAL_SAMPLES` variable. 

In [2]:
TOTAL_SAMPLES = 250
OUTPUT_DIR = './frames'

def get_nth_frame(cap: cv2.VideoCapture, n: int, n_frames: Optional[int] = None) -> np.ndarray: 
    """
    Retrieve the nth frame from a video capture object.

    This function extracts the nth frame from a given video capture object `cap`. 
    If `n_frames` is not provided, the function will determine the total number of frames 
    in the video. The frame counter is reset to its last value before the nth frame was 
    retrieved

    Parameters:
    cap (cv2.VideoCapture): The video capture object from which to extract the frame.
    n (int): The frame number to retrieve.
    n_frames (Optional[int], optional): The total number of frames in the video. 
                                        If not provided, it will be determined automatically.

    Returns:
    np.ndarray: The nth frame as an array.

    Raises:
    IndexError: If `n` exceeds the total number of frames available in the video.
    RuntimeError: If the frame could not be read from the video capture object.

    Example usage:
    >>> cap = cv2.VideoCapture('video.mp4')
    >>> frame = get_nth_frame(cap, 10)
    >>> cv2.imshow('10th Frame', frame)
    >>> cv2.waitKey(0)
    >>> cv2.destroyAllWindows()
    """
    n_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT) if n_frames is None else n_frames
    current_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)

    if n > n_frames: 
        raise IndexError(f"Attempted to get {n}th frame when only {n_frames} frames exist")
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, n)
    
    ret, frame = cap.read()
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)

    if not ret: 
        raise RuntimeError(f"Failed to read {n}th frame")
    
    return frame 


if not os.path.exists(OUTPUT_DIR) :
    os.mkdir(OUTPUT_DIR)
else: 
    raise KeyboardInterrupt("Un-annotated dataset already exists, skipping")

for i in trange(TOTAL_SAMPLES):
    n = np.random.randint(0, n_frames)
    frame = get_nth_frame(cap, n, n_frames=n_frames)
    cv2.imwrite(f"./frames/frame-{n}.jpg", frame)

KeyboardInterrupt: Un-annotated dataset already exists, skipping

## Annotation 

Please visit [roboflow](https://universe.roboflow.com/mice-maze/mice-maze) to view and download the annotated dataset. Ensure the dataset is in the same format as the following. 

    mice-maze-dataset
    ├── data.yaml
    ├── images
    │   ├── test
    │   ├── train
    │   └── val
    └── labels
        ├── test
        ├── train
        └── val

You may also need to edit the `data.yaml` file in order to get the data set working correctly. Here is an example of such a file 

```yaml
    path: /path/to/mice-maze-dataset
    train: images/train  # Path to training images
    val: images/val  # Path to validation images
    test: images/test  # Path to test images

    # Class names (you can add more if you have more classes)
    names:
        0: mouse
```

## Fine-tuning the model

In [3]:
# Select the best backend to train the model on 
# If no such backend is available, such as a CUDA-enabled 
# GPU, found in modern Nvidia cards, or MPS found in 
# Apple devices with an Apple Silicon CPU (M1, M2, etc), 
# make use of the CPU 
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_built():
    # NOTE: See https://github.com/ultralytics/ultralytics/issues/12999
    device = torch.device('mps')
else: 
    print("No backend availale, defaulting to CPU")
    print("This might take a while, please ensure you installed the correct version of pytorch for your hardware")
    print("Please see `https://pytorch.org/get-started/locally/`")
    device = torch.device('cpu')

if TRAIN: 
    yolo = ultralytics.YOLO('yolov8n.pt')
    add_wandb_callback(yolo, enable_model_checkpointing=True)
    yolo.train(data='mice-maze.yolov8/data.yaml', device=device)
else: 
    print("Model already trained, loading...")

Model already trained, loading...

# Real-World Performance of the Model 

In [4]:
trained_yolo = ultralytics.YOLO("/Users/henrywilliams/Documents/programming/python/mice-maze-rewrite/runs/detect/train30/weights/best.pt")

trained_yolo.track(VIDEO_PATH, persist=True, show=True)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/81367) /Users/henrywilliams/Downloads/videos/maze_test.mp4: 640x640 (no detections), 101.7ms
video 1/1 (frame 2/81367) /Users/henrywilliams/Downloads/videos/maze_test.mp4: 640x640 (no detections), 70.3ms
video 1/1 (frame 3/81367) /Users/henrywilliams/Downloads/videos/maze_test.mp4: 640x640 (no detections), 65.7ms
video 1/1 (frame 4/81367) /Users/henrywilliams/Downloads/videos/maze_test.mp4: 640x640 (no detections), 66.3ms
video 1/1 (

KeyboardInterrupt: 

: 